In [1]:
# pull ACMI creator relations from Wikidata.

import pandas
import pydash
import requests

def value_extract(row, column):

    ''' Extract dictionary values. '''
    
    return pydash.get(row[column], 'value')

def sparql_query(query, service):

    ''' Send sparql request, and formulate results into a dataframe. '''

    response = requests.get(service, params={'format': 'json', 'query': query}, timeout=120)
    results = pydash.get(response.json(), 'results.bindings')
    data_frame = pandas.DataFrame.from_dict(results)
    for column in data_frame.columns:
        data_frame[column] = data_frame.apply(value_extract, column=column, axis=1)
    
    return data_frame

query = '''
    select ?acmi_creator ?wikidata_creator ?wikidata_creatorLabel ?relation ?individual where {
        values ?relation {wdt:P26 wdt:P451 wdt:P40 wdt:P22 wdt:P25 wdt:P3373}
        ?wikidata_creator wdt:P7003 ?acmi_creator .
        filter(contains(str(?acmi_creator),'creator' )) 
        ?wikidata_creator ?relation ?individual .
        service wikibase:label { bd:serviceParam wikibase:language "en". }
        }'''

dataframe = sparql_query(query, 'https://query.wikidata.org/sparql')
dataframe = dataframe.loc[dataframe.individual.isin(list(dataframe.wikidata_creator))]

creator_key_wikidata_acmi = {x['wikidata_creator']:x['acmi_creator'] for x in dataframe.to_dict('records')} 
dataframe = dataframe.replace({'wikidata_creator':creator_key_wikidata_acmi, 'individual':creator_key_wikidata_acmi})

dataframe['acmi_id_1'] = dataframe['wikidata_creator']
dataframe['acmi_name_1'] = dataframe['wikidata_creator']

dataframe['acmi_id_2'] = dataframe['individual']
dataframe['acmi_name_2'] = dataframe['individual']

creator_key_acmi_name = {x['acmi_creator']:x['wikidata_creatorLabel'] for x in dataframe.to_dict('records')} 
dataframe = dataframe.replace({'acmi_name_1':creator_key_acmi_name, 'acmi_name_2':creator_key_acmi_name})

dataframe = dataframe.replace({'relation': {
    'http://www.wikidata.org/prop/direct/P26': 'spouse',
    'http://www.wikidata.org/prop/direct/P451': 'unmarried partner',
    'http://www.wikidata.org/prop/direct/P40': 'child',
    'http://www.wikidata.org/prop/direct/P3373': 'sibling',
    'http://www.wikidata.org/prop/direct/P22': 'father',
    'http://www.wikidata.org/prop/direct/P25': 'mother'
}})

dataframe = dataframe[['acmi_id_1', 'acmi_name_1', 'relation', 'acmi_id_2', 'acmi_name_2']]

print(len(dataframe))
dataframe.head()

235


,acmi_id_1,acmi_name_1,relation,acmi_id_2,acmi_name_2
27,creators/78964,Denise Robert,spouse,creators/27472,Denys Arcand
44,creators/72664,Akira Kurosawa,child,creators/72666,Hisao Kurosawa
119,creators/10138,Ingmar Bergman,unmarried partner,creators/79892,Liv Ullmann
145,creators/71108,Dino De Laurentiis,child,creators/86566,Raffaella De Laurentiis
154,creators/77041,Jacques Tourneur,father,creators/79314,Maurice Tourneur
